In [4]:
class Srs:
    def __init__(self, data, name=None, index=None):
        self.values = Arr(data)
        self.index = index or Idx(range(len(self.values)))
        assert len(self.values) == len(self.index)
        self.name = name or "new_series"

    def __len__(self):
        return len(self.values)
    
    def __iter__(self):
        return zip(self.index.values, self.values)
    
    def __str__(self):
        ret = f"{self.name}: \n"
        for idx, val in self: # This is now possible thanks to __iter__
            ret += str(idx).center(6)
            ret += str(val).center(6)
            ret += '\n'

        return ret

    __repr__ = __str__
    
    def __getattr__(self, item):
        if hasattr(self.values, item):
            return getattr(self.values, item)
        raise AttributeError
    
    def __getitem__(self, items):
        if items == slice(None):
            # To solve things like srs[:]
            return self
        items = listify(items)
        idx = []
        for item in items:
            assert item in self.index.mapping.keys(), f'{items} is not in the series index'
            idx.extend(self.index.mapping.get(item))

        idx = sorted(idx)
        vals = [self.values[i] for i in idx]
        new_index = [self.index[i] for i in idx] # This is a bit overhead, but will suffice for now
        return Srs(vals, name=self.name, index=Idx(new_index))

    
    def as_index(self):
        return Idx(self.values)

    
    def value_counts(self):
        idx = self.as_index()
        map_ = idx.mapping
        keys = []
        counts = []
        for key, indices in map_.items():
            keys.append(key)
            counts.append(len(indices))
        ret = Srs(counts, name=f'{self.name} value counts', index=Idx(keys))
        return ret
        
    def __add__(self, other):
        return series_apply(self, other, '__add__')
    def __sub__(self, other):
        return series_apply(self, other, '__sub__')
    def __mul__(self, other):
        return series_apply(self, other, '__mul__')
    def __truediv__(self, other):
        return series_apply(self, other, '__truediv__')
    
        
def series_apply(left, right, f_name):
    # Same index. We will not deal with not aligned concats
    f = getattr(left.values, f_name) # We now now gett so let's make it nicer
    res = f(right.values)
    return Srs(res, index=left.index)


def listify(val):
    if isinstance(val, list):
        return val
    else:
        return [val]